<h1> How I learned to stop worrying and love uncertainty </h1>

An introductory workshop on quantifying uncertainty in building simulation.

<b>author</b>: Parag Rastogi; <b>venue</b>: TU Delft, Delft, The Netherlands; <b>date</b>: 04 June, 2018.

Run each module one-by-one by either using the <kbd>run cell</kbd> button above or pressing <kbd>Ctrl + Enter</kbd> when a cell is selected. Modules like this one are `Markdown` modules, which is a kind of text-encoding language. These will not produce an output - instead you will see formatted text in the cell when you run one.


<h2>Initialisation</h2>

<ol>
<li>If you haven't read the wiki, I suggest starting there: https://github.com/paragrastogi/SimAud_2018_Uncertainty/wiki . 

<li> Download a folder of weather data here: https://drive.google.com/drive/folders/15hyS_Rg-DMxu05FkoS_K-NfFdAozvgJK?usp=sharing . You will see two folders in there: `ddn` and `gen`. Place them in the top-level directory, i.e., at the same level as `lib`.

</ol>

Everything should work ok on a Mac, though I have only tested the exercises on Windows and Ubuntu. 

In [1]:
# Import a couple of packages to display images in the notebook.

from IPython.display import Image
from IPython.core.display import HTML 

<h2>Uncertainty in Building Simulation</h2>

<h3>What is it?</h3>

Building Performance Simulation is an estimate of the (thermal and electrical) energy performance of a building. Since the performance estimate relies on knowing many inputs, it is affected by lack of knowledge or randomness in these inputs. Thus, uncertainty can be separated into two types:

<ol>
    <li> Epistemic
    <li> Aleatory
</ol>
            


Using "Building Performance Simulation (BPS)... designers can obtain reliable performance estimates by testing their designs under many plausible operating conditions, e.g., the weather and usage the building might experience in the future. These estimates can then be used to choose a design that could deliver high performance for the rest of its life... Unfortunately, multiple simulations are time-consuming... Standard averaging methods, such as the Monte Carlo method, typically require a large number of simulations to ensure the quality of the estimate (MacDonald  2002; Iaccarino 2008}. In a typical building-design problem where a designer might test hundreds of designs, such methods might require hundreds of hours to run and quickly become infeasible. A preferred practice is to simply use a single 'average' or 'typical' estimate of future conditions, which is much faster. The danger of such a procedure is that it might miss a harmful operating condition where the building performs poorly or even breaks down. To ensure the robustness of designs it is, therefore, better to test them under a wide variety of plausible operating conditions. So is it possible to reduce the computation time of BPS such that multiple simulations during the design process are feasible?" (Rastogi, Khan, and Andersen 2018, <i>in review</i>)

Yes it is feasible! With rapid-response regression model, which we call an 'emulator' (because it emulates the original, physics-based building performance simulation model). A regression model can be used to answer some questions about a building's response, such as in robust design.




In [2]:
# Import various standard modules.
import glob
import os
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns

# Get inline graphs .
%matplotlib inline

# Only useful for debugging, when you 
# need to reload external modules.
from importlib import reload

# Enable xkcd mode - if you're a nerd.
# plt.xkcd()

# Import a custom read-write function for weather files.
import wfileio as wf

# import small helpers I've written.
import petites as petite

# Import an awesome colour palette. Call it colours.
import default_colours as colours

# Set the random seed to a specific value so
# the experiment is repeatable.
# See https://en.wikipedia.org/wiki/Random_seed
# for more information on what this means.
np.random.seed(42)

In [3]:
# Read Dehradun and Geneva location data.

# We will store some metadata about the stations as well.
metadata = dict(locdata=list(), header=list())

# We will only use the EPW file format for this exercise.
file_type = 'epw'

# I use os.path.join so the paths are automatically constructed based on the OS.
path_epw_ddn = os.path.join('..', 'ddn', 'IND_UT_Dehradun.421110_ISHRAE2014.epw')

# The small program get_weather stores data from the incoming weather
# file as a dataframe.
ddn, locdata, header = wf.get_weather('ddn', path_epw_ddn, file_type=file_type)

# Save some metadata about the location.
metadata['locdata'].append(locdata)
metadata['header'].append(header)

# Do the same with Geneva data. Except Geneva has multiple weather files, so use a loop.
path_gen_folder = os.path.join('..', 'gen')
list_genfiles = glob.glob(os.path.join(path_gen_folder, '*.{:s}'.format(file_type)))

# Declare a list.
gen = list()

for file in list_genfiles:
    
    gen_temp, locdata, header = wf.get_weather('gen', file, file_type=file_type)
    gen.append(gen_temp)
    
gen = pd.concat(gen)

metadata['locdata'].append(locdata)
metadata['header'].append(header)

# Set the variable 'tmy' to the incoming data from Dehradun - so changing 
# this statement will allow you to run the worksheet with any location.
tmy = ddn

ValueError: No objects to concatenate

In [ ]:
print('DEHRADUN')

print(ddn.describe())

print('======\r\n')

print('GENEVA')

print(gen.describe())

print('======')


<h2> Black Box Models </h2>

For this exercise we will use the equation presented in Clarke (2001, Chpt. 7, sec. 7.1, pg. 206):

$ E = a \theta^2 + b R_d^2 + c R_f^2 + d V^2 + e \theta + f R_d + g R_f + h V + i \theta R_d + j \theta R_f + k \theta V + l R_d R_f + m R_d V + n R_f V + o, \quad$  (1)

where "$E$ is the monthly energy requirement \[kWh\], $\theta$ the monthly mean temperature \[$^o$C\], $R_d$ the monthly mean direct normal solar radiation \[W/$m^2$\], $R_f$ the monthly mean diffuse horizontal solar radiation \[W/$m^2$\], $V$ the montly mean wind speed [m/s], and 'a' through 'o' are the least squares coefficients" (ibid).

<b>NB</b>: The equation in the code is the same as this one, I've just rearranged the parameters and stated the constants as elements of an array rather than as separate variables. Also, I have omitted the bars over the variable names since the markdown rendering of bars and powers is awkward.


<h2>Exercises</h2>

We will do two exercises in this workshop: 

<ol>
<li> Fixed inputs, random coefficients: 
<li> Random inputs, fixed coefficients:
</ol>

In [ ]:
# Define a function to calculate monthly energy use using the equation given in Clarke (2001).

def E_month(theta, R_d, R_f, V, beta=np.random.rand(15,1)):

    # By default, this function will use random coefficients to calculate the energy use. 
    # This is just for a demo since the coefficients are never 
    # going to be 'random', unless you have no idea what you are doing.
            
    e = beta[0] + beta[1]*theta + beta[2]*R_d + beta[3]*R_f + beta[4]*V + \
            beta[5]*theta**2 + beta[6]*R_d**2 + beta[7]*R_f**2 + beta[8]*V**2 + \
            beta[9]*theta*R_d + beta[10]*theta*R_f + beta[11]*theta*V + \
            beta[12]*R_d*R_f + beta[13]*R_d*V + beta[14]*R_f*V
    
    return e


## Random coefficients, fixed inputs

For the first exercise, let's fix the inputs to typical values and use random coefficients to specify the relationship between inputs and output. 

The simulators you use will almost never be random - for buildings at least - so just treat this as an exercise. Imagine the building is changing, so its response to the weather inputs is changing.

In [ ]:
# Calculate the monthly mean values of weather parameters for Dehradun.

relevant_keys = ['tdb', 'dni', 'ghi', 'wspd']

# Declare an empty dataframe.
tmy_summary = pd.DataFrame()

# Assign individual columns.
# Each weather parameter is grouped by month, and the monthly mean calculated.
tmy_summary['month'] = pd.unique(ddn['month'])
tmy_summary.index = tmy_summary['month']

for key in tmy.columns:
    if key not in relevant_keys:
        continue
    tmy_summary[key] = tmy.groupby(by=['month'])[key].mean()

# Calculate the monthly energy consumption with random coefficients.
tmy_summary['energy'] = E_month(tmy_summary['tdb'], tmy_summary['dni'],
                                       tmy_summary['ghi'], tmy_summary['wspd'])

tmy_summary.head()

In [ ]:
# Do groupby manually using a for loop.

# You don't need this - this is just a demo to show you how much easier your life is with pandas!

tmy_summary2 = np.zeros_like(tmy_summary)

for month in tmy_summary.index:
    mask = ddn['month'] == month
    tmy_summary2[month-1, tmy_summary.columns=='month'] = month
    for key in tmy.columns:
        if key not in relevant_keys:
            continue
        tmy_summary2[month-1, tmy_summary.columns==key] = np.mean(tmy.loc[mask, key])

        # End inner for loop.
# End outer for loop.    

tmy_summary2 = pd.DataFrame(data=tmy_summary2,
                            index=pd.unique(ddn['month']),
                            columns=tmy_summary.columns)

tmy_summary2['energy'] = E_month(tmy_summary2['tdb'], tmy_summary2['dni'],
                                        tmy_summary2['ghi'], tmy_summary2['wspd'])

print('The two dataframes should match, so most of the differences should be close to zero.')

In [ ]:
diffs = tmy_summary2.values-tmy_summary.values
print('Median = {0:4.3e}; Max. = {1:4.3e}; Min = {2:4.3e}.'.format(np.median(diffs), np.max(diffs), np.min(diffs)))

In [ ]:
# Try smoothing instead of using a monthly average.

def smoother(xin, lags):
    
    xout = np.zeros_like(xin)
    
    # Convert even numbers to odd.
    if np.mod(lags, 2) == 0:
        lags = lags + 1
        
    halfspan = int(np.floor(lags/2))
        
    for n, x in enumerate(xin):
        
        if (n < halfspan):
            xout[n] = np.nanmean(xin[0:halfspan])
        
        elif (n > (xin.shape[0] - halfspan)):
            xout[n] = np.nanmean(xin[-halfspan:-1])
        
        else:
            lb = int(n - halfspan)
            ub = int(n + halfspan)
            xout[n] = np.nanmean(xin[lb:ub])
        
        # End IF statement.
        
    # End FOR loop.
    
    return xout

We're going to use the smoothed data to calculate energy use but be aware that the original regression relationship was created with *monthly values*.

In [ ]:
span = 720

ddn_smooth = pd.DataFrame(data = np.zeros(tmy.shape), columns=tmy.columns, index=tmy.index)

for key in tmy.columns:
    if key not in relevant_keys:
        continue
    ddn_smooth[key] = smoother(ddn[key].values, span)
    
ddn_smooth['energy'] = E_month(ddn_smooth['tdb'], ddn_smooth['dni'],
                                       ddn_smooth['ghi'], ddn_smooth['wspd'])

In [ ]:
plot_x = np.arange(0, tmy.shape[0])
# Find the index and value of the max temperature.
ymax = np.max(ddn['tdb'].values)
xmax = plot_x[ddn['tdb'].values==ymax]
ymin = np.min(ddn['tdb'].values)
xmin = plot_x[ddn['tdb'].values==ymin]

fig, ax = plt.subplots(figsize=[12,6])
ax.plot(plot_x, ddn_smooth['tdb'].values, zorder=10, color=colours.blue)
ax.plot(plot_x, ddn['tdb'].values, zorder = 1, color=colours.orange, alpha=0.5)
ax.plot(plot_x, np.repeat(tmy_summary['tdb'], 8760/12), color=colours.blackest, zorder=11)
plt.legend(['Hourly', 'Smoothed', 'Monthly'])
plt.ylabel('Dry bulb temperature [$^o$C]')
plt.xlabel('Hour of the year')
plt.xlim(plot_x[0], plot_x[-1])
plt.annotate('The hottest temperature \n in the TMY file\n',
    xy=(xmax, ymax), arrowprops=dict(arrowstyle='->'), xytext=(4000, 10))
plt.annotate('The coldest temperature \n in the TMY file\n',
    xy=(xmin, ymin), arrowprops=dict(arrowstyle='->'), xytext=(4000, 5))
plt.show()


In [ ]:
plot_x = range(0, tmy.shape[0])
fig, ax = plt.subplots(figsize=[12,6])
ax.plot(plot_x, ddn_smooth['energy'], alpha=1, zorder=3, color=colours.blue)
ax.plot(plot_x, np.repeat(tmy_summary['energy'], 8760/12), color=colours.blackest, zorder=10)
plt.legend(['hourly', 'monthly'])
plt.ylabel('Energy [kWh]')
plt.xlabel('Hour of the year')
plt.show()

In [ ]:
# Repeat the above exercise with data from Ahmedabad.


In [ ]:
# Plot comparisons of weather from Dehradun and Ahmedabad.
# Realise Dehradun is a far superior place to live than Ahmedabad. 


In [ ]:
fig, ax = plt.subplots(figsize=[12,6])
ax.scatter(tmy_summary['tdb'], tmy_summary['energy'], color=colours.blue)
plt.ylabel('Energy [kWh]')
plt.xlabel('Temperature [$^o$C]')
plt.title('Energy use vs Temperature')
plt.show()

fig, ax = plt.subplots(figsize=[12,6])
ax.plot(range(0, tmy_summary.shape[0]), tmy_summary['energy'], color=colours.blue)
plt.ylabel('Energy [kWh]')
plt.xlabel('Month')
plt.title('Energy use per month with random coefficients')
plt.xticks(range(0, 12))
plt.show()

## Fixed coefficients, random inputs

Now let's try the same exercise with fixed coefficients and random inputs.


In [ ]:
# import the Python module that can read these kinds of files.
import h5py

path_to_synw = os.path.join('..', 'gen', 'synthetic', 'syn_2051_2060.p')

# The column names we expect to see in the synthetic files.
column_names = ('year', 'month', 'day', 'hour', 'tdb', 'tdp', 'rh',
                'ghi', 'dni', 'dhi', 'wspd', 'wdr')
dc = 4  # No. of date columns.

In [ ]:
# Make a variable to point to the file.
f = h5py.File(path_to_synw)
# Specify an empty dataframe.
syndata = pd.DataFrame()

# Take each 'key' (variable) from the file and assign it to syndata.
for key in f['syndata']:
    syndata[key.lower()] = pd.Series(np.squeeze(np.array(f['syndata'][key])))

f.close()

# Number of years in dataset.
n_years = np.unique(syndata['year']).size
n_hours = 8760  # Fixed for this dataset.

In [ ]:
# Add wspd key.
syndata['wspd'] = pd.Series(data=np.repeat(ddn['wspd'].values, n_years), index=syndata.index)
syndata['wdr'] = pd.Series(data=np.repeat(ddn['wdr'].values, n_years), index=syndata.index)
syndata['tdp'] = pd.Series(data=calc_tdp(syndata['tdb'].values, syndata['rh'].values), index=syndata.index)

In [ ]:
# Rearrange the date columns of this dataframe.
# The columns should have been in this order already
# but something was probably wrong in the reading/writing
# or generation of the synthetic file.

cols = syndata.columns.tolist()
# print(ddn.columns)
myorder = [3, 2, 0, 1, 8, 11, 7, 6, 5, 4, 9, 10]
cols = [cols[i] for i in myorder]
syndata = syndata[cols]
print(syndata.columns)

In [ ]:
# Now let's fix the coefficients to some arbitrary values and plot inputs from  

span = 720

syn_smooth = pd.DataFrame(data = np.zeros(syndata.shape), columns=syndata.columns, index=syndata.index)

for key in syndata.columns:
    if key not in relevant_keys:
        syn_smooth[key] = syndata[key].values
    else:
        syn_smooth[key] = smoother(syndata[key].values, span)
    
syn_smooth['energy'] = E_month(syn_smooth['tdb'], syn_smooth['dni'],
                               syn_smooth['ghi'], syn_smooth['wspd'],
                               beta=np.ones([15, 1]))

In [ ]:
# It is generally not good practice to load modules after some code has been written.
# If I were to write this as a program, I would shift these calls to the top.
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import FormatStrFormatter

In [ ]:
x = range(0, n_hours)
y = range(0, n_years)

# Line plot.
fig = plt.figure(figsize=(8, 6), facecolor='w', edgecolor='k')
ax = fig.gca(projection='3d')

X, Y = np.meshgrid(x,y)
Z = syn_smooth['tdb'].values.reshape(-1, n_hours)

surf = ax.plot_surface(X, Y, Z, cmap=cm.cool,
                       linewidth=0, antialiased=False)

ax.set_xlim(0, n_hours)
ax.set_ylim(y[0], y[-1])
ax.set_zlim(np.min(Z), np.max(Z))

ax.set_ylabel('Year')
ax.set_xlabel('Hours')
ax.set_zlabel('Temperature [$^o$C]')

plt.show()

In [ ]:
x = range(0, n_hours)
y = range(0, n_years)

# Line plot.
fig = plt.figure(figsize=(8, 6), facecolor='w', edgecolor='k')
ax = fig.gca(projection='3d')

X, Y = np.meshgrid(x,y)
Z = syn_smooth['energy'].values.reshape(-1, n_hours)

surf = ax.plot_surface(X, Y, Z, cmap=cm.cool,
                       linewidth=0, antialiased=False)

ax.set_xlim(0, n_hours)
ax.set_ylim(y[0], y[-1])
ax.set_zlim(np.min(Z), np.max(Z))

ax.set_ylabel('Year')
ax.set_xlabel('Hours')
ax.set_zlabel('Energy [kWh]')

ax.set_zticks(np.arange(100000, 500000, 100000), )

plt.show()

In [ ]:
x = range(0, n_hours)
y = range(0, n_years)

# Line plot.
fig = plt.figure(figsize=(8, 6), facecolor='w', edgecolor='k')
ax = fig.gca()

# X, Y = np.meshgrid(x,y)
# z = syn_smooth['energy'].values.reshape(-1, n_hours)

bp = sns.boxplot(x='month', y='energy', data=syn_smooth, palette='Set3')

# ax.set_xlim(0, n_hours)
# ax.set_ylim(y[0], y[-1])
# ax.set_zlim(np.min(Z), np.max(Z))

ax.yaxis.set_major_formatter(FormatStrFormatter('%4.0e'))
ax.xaxis.set_major_formatter(FormatStrFormatter('%02d'))

# ax.set_ylabel('Year')
ax.set_xlabel('Month')
ax.set_ylabel('Energy [kWh]')

plt.show()

# Solar power with random weather inputs

## Optional

Use the `solar_power_func` to plot solar power production with random inputs. This function uses the procedure laid out in the iPython notebook: `solar_power`. 

I'm having trouble writing out the synthetic data to an EPW file for the `solar_power` function to read, but if you're interested we can talk about this.

Bonus exercise: optimise tilt and/or orientation angle.

In [ ]:
import solar_power_func as solar

In [ ]:
power = solar.tmy_to_power(path_tmy_data=path_epw_ddn,
                           surface_tilt=30, surface_azimuth=180,
                           albedo=0.2, silent=True)

In [ ]:
plot_x = np.arange(0, tmy.shape[0])

fig, ax = plt.subplots(figsize=[12,6])
ax.plot(plot_x, power, zorder = 1, color=colours.orange, alpha=0.5)
plt.ylabel('AC power [W]')
plt.xlabel('Hour of the year')
plt.xlim(plot_x[0], plot_x[-1])
plt.show()

In [ ]:
ts = syndata.iloc[0:n_hours, :].values
fpath_out = os.path.join("..", "ddn", "ddn_syn.epw")
wf.give_weather(ts, locdata=locdata, stcode='ddn', header=header,
                 masterfile=path_epw_ddn, ftype='epw',
                 s_shift=0, fpath_out=fpath_out)

In [ ]:
ddn_syn, locdata_syn, header_syn, _ = wf.get_weather('ddn_syn', fpath_out, ftype='epw')

In [ ]:
power_syn = solar.tmy_to_power(path_tmy_data=fpath_out,
                           surface_tilt=30, surface_azimuth=180,
                           albedo=0.2, silent=True)

In [ ]:
plot_x = np.arange(0, tmy.shape[0])

fig, ax = plt.subplots(figsize=[12,6])
ax.plot(plot_x, power_syn, zorder = 1, color=colours.orange, alpha=0.5)
plt.ylabel('AC power [W]')
plt.xlabel('Hour of the year')
plt.xlim(plot_x[0], plot_x[-1])
plt.show()

## Uncertainty analysis with EPlus

### Steps/Notes

1. Consider weather inputs from Indra.
2. Run samples using EPlus - analyse results.
    1. Either modify a base EPlus file using Eppy/scripting or in jePlus.
    2. Or analyse the same EPlus file using many weather files.

<h2> Bibliography </h2>

<ul>
<li>Rastogi, Parag. 2016. “On the Sensitivity of Buildings to Climate: The Interaction of Weather and Building Envelopes in Determining Future Building Energy Consumption.” PhD, Lausanne, Switzerland: Ecole polytechnique fédérale de Lausanne. EPFL Infoscience. https://infoscience.epfl.ch/record/220971?ln=en.
<li>Rastogi, Parag, and Marilyne Andersen. 2015. “Embedding Stochasticity in Building Simulation Through Synthetic Weather Files.” In Proceedings of BS 2015. Hyderabad, India. http://infoscience.epfl.ch/record/208743.
<li>———. 2016. “Incorporating Climate Change Predictions in the Analysis of Weather-Based Uncertainty.” In Proceedings of SimBuild 2016. Salt Lake City, UT, USA. http://infoscience.epfl.ch/record/208743.
<li>Rastogi, Parag, Mohammad Emtiyaz Khan, and Marilyne Andersen. 2017. “Gaussian-Process-Based Emulators for Building Performance Simulation.” In Proceedings of BS 2017. San Francisco, CA, USA: IBPSA.
<li>Iaccarino, Gianluca. 2008. “Quantification of Uncertainty in Flow Simulations Using Probabilistic Methods.” presented at the VKI Lecture Series, Stanford University, September. http://web.stanford.edu/group/uq/uq_youq.html.
<li>Macdonald, Iain. 2002. “Quantifying the Effects of Uncertainty in Building Simulation.” Doctoral, University of Strathclyde. https://www.strath.ac.uk/media/departments/mechanicalengineering/esru/research/phdmphilprojects/macdonald_thesis.pdf.

</ul>